In [5]:
from datasets import load_dataset
import requests
import json
from supabase import create_client, Client
import time
import os
import openai
import dotenv
dotenv.load_dotenv()
import tqdm 
import pandas as pd
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 
print(supabase_url)

https://zioyzzneyeglzxlgtdrw.supabase.co


In [6]:
# Load the dataset, initializa supabase client
supabase: Client = create_client(supabase_url, supabase_api_key)


```sql
create table multidoc_sources (
  id bigint primary key generated always as identity,
  docid bigint,
  dataType text,
  title text,
  content text, 
  embedding vector(1536)
);
```

In [4]:
model_name = "text-embedding-3-small"
def get_embeddings(input, model):
  payload = { "input": input, "model": model }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else:
    time.sleep(3)
    print("embedding error..... retrying")
    # retry
    return get_embeddings(input, model)


In [8]:
# 資料前處理
source = pd.read_csv("sources.csv")
contents = source["content"].to_list()
max_len =  -1
for i in contents:
    max_len = max(max_len, len(i))
print(max_len)

1998


In [9]:
print(len(contents[1]))

1062


In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=250,
)
# test
result = text_splitter.split_text(contents[1])
for i in result:
    print(len(i))

390
396
372
390
399
208


In [19]:
source

,id,dataType,title,content
0,154437,百科,A100式火箭炮,同義詞 中國A100式300毫米自行火箭炮一般指A100式火箭炮 A100型300毫米多管火...
1,561731,百科,VIX,VIX 是芝加哥期權期貨交易所 使用的市場波動性指數。通過該指數，可以瞭解到市場對未來30天...
2,47848,百科,hi0141,"hi0141是一種生物,屬於植物界,選育單位是先正達種子公司。 基本信息 審定編號:蒙審甜號..."
3,200746,百科,phiaton,Phiaton品牌2008年於美國註冊，同年在美國CES展會上正式推出，目前正以美國爲中心向...
4,4677,百科,一三慘案,一三慘案是指1927年1月3日，武漢工人、學生和市民召開大會，慶祝北伐勝利和國民政府遷到武漢...
...,...,...,...,...
995,1346606,百科,涼拌芥藍,涼拌芥藍是一道簡單的家常菜，主料是芥藍。涼拌芥藍製作方法： [1] 主料：芥藍 配料：枸杞、...
996,1193065,百科,歷代最經典花鳥畫,《歷代最經典花鳥畫》是2012年浙江人民美術出版社出版的圖書，由名畫再現編委會編著。本書精選...
997,1242925,百科,反病毒監測,瑞星全球反病毒監測網也截獲了全球首個WindowsLinux雙系統病毒，據瑞星反病毒專家分析...
998,1349450,百科,同態反褶積,同態反褶積（homomorphic deconvolution）是一種非線性濾波方法。它應用...


In [26]:
def process_documents(source):
    """
    處理文檔並生成嵌入向量，包含斷點續傳機制
    
    Args:
        source: 包含文檔資料的DataFrame
    """
    # 檢查已處理的文檔
    def get_processed_docs(supabase):
        response = supabase.table('multidoc_sources').select('docid').execute()
        return set(item['docid'] for item in response.data)
    
    try:
        # 獲取已處理的文檔ID
        processed_docs = get_processed_docs(supabase)
        
        for index, row in source.iterrows():
            row_id = row["id"]
            
            # 跳過已處理的文檔
            if row_id in processed_docs:
                print(f"跳過已處理的文檔: {row['title']}")
                continue
                
            row_datatype = row["dataType"]
            row_title = row["title"]
            row_content = row["content"]
            
            try:
                # 分塊處理內容
                chunked_content = text_splitter.split_text(row_content)
                print(f"處理文檔: {row_title}")
                
                for chunk in chunked_content:
                    embed_chunk = row_title + " " + chunk
                    embedding = get_embeddings(embed_chunk, model_name)
                    
                    # 插入資料庫
                    supa_response = supabase.table('multidoc_sources').upsert({
                        "docid": row_id,
                        "datatype": row_datatype,
                        "title": row_title,
                        "content": embed_chunk,
                        "embedding": embedding
                    }).execute()
                    
                print(f"完成處理文檔: {row_title}")
                
            except Exception as e:
                print(f"處理文檔 {row_title} 時發生錯誤: {str(e)}")
                continue
                
    except Exception as e:
        print(f"程式執行發生錯誤: {str(e)}")
        raise

# 使用方式
# process_documents(source_dataframe)

In [27]:
process_documents(source)


跳過已處理的文檔: A100式火箭炮
跳過已處理的文檔: VIX
跳過已處理的文檔: hi0141
跳過已處理的文檔: phiaton
跳過已處理的文檔: 一三慘案
跳過已處理的文檔: 一丘一壑
跳過已處理的文檔: 一個人失眠
跳過已處理的文檔: 一個人的折磨
跳過已處理的文檔: 一個人遠方
跳過已處理的文檔: 一個奶奶三個綁
跳過已處理的文檔: 一個月的愛情
跳過已處理的文檔: 一個船伕的故事
跳過已處理的文檔: 一九
跳過已處理的文檔: 一代英豪
跳過已處理的文檔: 一體芯插座
跳過已處理的文檔: 一兜糖
跳過已處理的文檔: 一分錢一分貨
跳過已處理的文檔: 一切險合同
跳過已處理的文檔: 一包中華
跳過已處理的文檔: 一半的我
跳過已處理的文檔: 一口英語
跳過已處理的文檔: 一隻狼
跳過已處理的文檔: 一號病
跳過已處理的文檔: 一品臺
跳過已處理的文檔: 一唯
跳過已處理的文檔: 一塊石頭落地
跳過已處理的文檔: 一夜後婚
跳過已處理的文檔: 一字記之曰日
跳過已處理的文檔: 一寸丹心
跳過已處理的文檔: 一尾概率
跳過已處理的文檔: 一年三班惡魔軍團
跳過已處理的文檔: 一座燈塔
跳過已處理的文檔: 一心亭
跳過已處理的文檔: 一房一驗
跳過已處理的文檔: 一抹綠
跳過已處理的文檔: 一攬子個稅
跳過已處理的文檔: 一日一字
跳過已處理的文檔: 一曲成名
跳過已處理的文檔: 一本無限流小說
跳過已處理的文檔: 一條繩倆螞蚱
跳過已處理的文檔: 一柱通天
跳過已處理的文檔: 一夢經年
跳過已處理的文檔: 一次性使用集尿袋
跳過已處理的文檔: 一次燒微晶
跳過已處理的文檔: 一段話作文
跳過已處理的文檔: 一汽解放軸齒中心
跳過已處理的文檔: 一滴甲
跳過已處理的文檔: 一片無痕文胸
跳過已處理的文檔: 一生三秋
跳過已處理的文檔: 一生有夢
跳過已處理的文檔: 一界
跳過已處理的文檔: 一瞬芳華
跳過已處理的文檔: 一積
跳過已處理的文檔: 一筆畫遊戲攻略
跳過已處理的文檔: 一類治超站
跳過已處理的文檔: 一級維護
跳過已處理的文檔: 一維無限深勢阱
跳過已處理的文檔: 一腳超人
跳過已處理的文檔: 一般式
跳過已處理的文檔: 一色邊
跳過已處理的文檔: 一覽表
跳過已處理的文檔: 一貫道
跳過已處理的文檔: 一起來洗澡
跳過已處理的文檔: 一路上的風景

```sql

create table multidoc_sources (
  id bigint primary key generated always as identity,
  docid bigint,
  dataType text,
  title text,
  content text, 
  embedding vector(1536)
);

create table multidoc_questions (
  id bigint primary key generated always as identity,
  question text,
  answer text,
  docid bigint,
  dataType text,
  question_embedding vector(1536),
  answer_embedding vector(1536)
);
```

In [35]:
def process_documents(source):
    """
    處理文檔並生成嵌入向量，包含斷點續傳機制
    
    Args:
        source: 包含文檔資料的DataFrame
    """
    # 檢查已處理的文檔
    def get_processed_docs(supabase):
        response = supabase.table('multidoc_questions').select('docid').execute()
        return set(item['docid'] for item in response.data)
    
    try:
        # 獲取已處理的文檔ID
        processed_docs = get_processed_docs(supabase)
        
        for index, row in source.iterrows():
            row_docid = row["docid"]
            
            # 跳過已處理的文檔
            if row_docid in processed_docs:
                print(f"跳過已處理的文檔: {row['title']}")
                continue
                
            row_datatype = row["dataType"]
            row_question = row["question"]
            row_answer = row["answer"]
            
            try:
                # 分塊處理內容
                
                question_embedding = get_embeddings(row_question, model_name)
                answer_embedding = get_embeddings(row_answer, model_name)
                
                # 插入資料庫
                supa_response = supabase.table('multidoc_questions').upsert({
                    "docid": row_docid,
                    "datatype": row_datatype,
                    "question": row_question,
                    "answer": row_answer,
                    "question_embedding": question_embedding,
                    "answer_embedding": answer_embedding
                }).execute()
                    
                print(f"完成處理文檔第{index}篇: {row_question[0:10]}...")
                
            except Exception as e:
                print(f"處理文檔 {index} 時發生錯誤: {str(e)}")
                break
                
    except Exception as e:
        print(f"程式執行發生錯誤: {str(e)}")
        raise

# 使用方式
# process_documents(source_dataframe)

In [33]:
# 資料前處理
question = pd.read_csv("questions.csv")
contents = question["question"].to_list()
max_len =  -1
for i in contents:
    max_len = max(max_len, len(i))
print(max_len)


52


In [36]:
process_documents(question)

完成處理文檔第0篇: A100式火箭炮的基...
完成處理文檔第1篇: VIX指數的作用是什...
完成處理文檔第2篇: Phiaton是哪個...
完成處理文檔第3篇: 1927年1月3日一...
完成處理文檔第4篇: 一丘一壑的成語來源於...
完成處理文檔第5篇: 張謙卑的歌曲《一個人...
完成處理文檔第6篇: 任珅是哪首歌的演唱者...
完成處理文檔第7篇: 許夢遠和那個陽光帥氣...
完成處理文檔第8篇: 電影《一個奶奶三個綁...
完成處理文檔第9篇: 尹瑩和徹宇分手的原因...
完成處理文檔第10篇: 《一個船伕的故事》的...
完成處理文檔第11篇: 在《周易參同契》中，...
完成處理文檔第12篇: 電影《一代英豪》講述...
完成處理文檔第13篇: 一體芯插座的優點有哪...
完成處理文檔第14篇: 一兜糖APP的主要功...
完成處理文檔第15篇: 孫博的歌曲《一分錢一...
完成處理文檔第16篇: 一切險合同指的是什麼...
完成處理文檔第17篇: 《一包中華》這部網絡...
完成處理文檔第18篇: 范瑋琪的《一半的我》...
完成處理文檔第19篇: 一口英語Englis...
完成處理文檔第20篇: 華聲兒童合唱團演唱的...
完成處理文檔第21篇: 鼠疫是什麼？...
完成處理文檔第22篇: 一品臺的總戶數是多少...
完成處理文檔第23篇: 什麼是一唯？...
完成處理文檔第24篇: 爲什麼趙溫聽說大人不...
完成處理文檔第25篇: 網絡小說《一夜後婚》...
完成處理文檔第26篇: 《一字記之曰日》是誰...
完成處理文檔第27篇: 成語“一寸丹心”的出...
完成處理文檔第28篇: 什麼是一尾概率？...
完成處理文檔第29篇: 該小說的作者是誰？...
完成處理文檔第30篇: 一座燈塔的光照亮了什...
完成處理文檔第31篇: 一心亭餐館的特色服務...
完成處理文檔第32篇: 建設單位委託第三方中...
完成處理文檔第33篇: 《一抹綠》這部網絡小...
完成處理文檔第34篇: 什麼是一攬子個稅改革...
完成處理文檔第35篇: 《一日一字》適合於哪...
完成處理文檔第36篇: 英文電影《一曲成名》...
完成處理文檔第37篇: 《一本無限流小說》的...
完成處理文檔第38篇: 王宣唱的《一條繩倆螞...
完成處理文檔第